In [125]:
import altair as alt
import pandas as pd

In [126]:
df = pd.read_csv("data.csv", encoding='UTF-16 LE', sep=",")
df.head()

,Регіон,рік,сільськегосподарство,промисловість,будівництво,послуги,державнеуправління
0,Україна,2010,8.3,25.9,3.3,57.3,5.2
1,Україна,2017,12.1,25.2,2.5,53.7,6.5
2,Вінницька,2010,21.6,19.3,4.1,47.4,7.6
3,Вінницька,2017,28.1,21.4,2.0,39.8,8.7
4,Волинська,2010,16.9,15.6,3.2,57.7,6.6


In [127]:
df_tall = pd.melt(df, id_vars=['Регіон', 'рік'], value_vars=['сільськегосподарство','промисловість', 'будівництво', 'послуги',  'державнеуправління',])
df_tall.head()

,Регіон,рік,variable,value
0,Україна,2010,сільськегосподарство,8.3
1,Україна,2017,сільськегосподарство,12.1
2,Вінницька,2010,сільськегосподарство,21.6
3,Вінницька,2017,сільськегосподарство,28.1
4,Волинська,2010,сільськегосподарство,16.9


Недоліки оригінальної візуалізації: 
    через використаня stacked bar chart-у нормальо порівнюються тільки значення для сільського господарства
    дані для 2017 і 2010 років розташовані поряд, через що важко порівнювати дані між областями
    категорії ВДВ закодовані кольором, через що потрібо постійно відволікатися на легеду
    розподіл в межах області теж не дуже легко зчитується


Оскільки на візуалізації занадто багато даних, будь-який спосіб її спростити буде концентруватись на одному аспекті даних і відкидати інші. Оригінальна візуалізація названа "Зміна структури ВДВ у розрізі регіонів за 2017 рік у порівнянні з 2010 роком", тож зосередитись треба на власне зміні структури для кожної області, тоді як різниця між областями є менш важливою.
Хорошим способом візуалізації зміни структури, на мою думку, є bar chart, в якому стовпчики для двох років накладаються один на одного. З нього легко зчитати як зміну кожної катогорії, так і різницю між різними катеоріями. Візуалізації для різних регіонів можна було б розділити з використанням інтерактивності(наприклад dropdown list), але оскільки ориінальна візуалізація інтерактивності не підтримує, я вирішив обійтися фасетингом. Графік по Україні показує принципоо інші дані, тому я відокремив його від решти.

In [128]:
bars = alt.Chart(df_tall[df_tall['Регіон']!='Україна']).mark_bar(opacity=0.9).encode(
                y=alt.Y('variable:N',
                    sort=alt.Sort(field='value', op='max', order='descending')
                ),
                x=alt.X('value:Q', stack=False),
                color=alt.Color('рік:N'),
                order=alt.Order('value:Q', aggregate='sum',sort='descending'),
            )

bars.properties(
                width=900/5, height=100
            ).facet(
                facet=alt.Facet('Регіон:N', sort=alt.Sort(field='value', op='max', order='descending'),title=''), columns=5
          )


alt.FacetChart(...)

In [131]:
bars_total = alt.Chart(df_tall[df_tall['Регіон']=='Україна']).mark_bar(opacity=0.9).encode(
                x=alt.X('variable:N',
                    sort=alt.Sort(field='value', op='max', order='descending'), title=''
                ),
                y=alt.Y('value:Q', stack=False, title=''),
                color=alt.Color('рік:N'),
                order=alt.Order('value:Q', aggregate='sum',sort='descending'),
            )
bars_total.properties(width=200, height=500)


alt.Chart(...)

In [132]:
bars.properties(
                width=700/5, height=100
            ).facet(
                facet=alt.Facet('Регіон:N', sort=alt.Sort(field='value', op='max', order='descending')), columns=5
          ) | bars_total.properties(width=200, height=500)


alt.HConcatChart(...)

Альтернативним способом візуалізації може бути гантельковий графік, але він менш зрозумілий, a малі зміни читаються з нього набагато гірше. 

In [134]:
__data = df_tall[df_tall['Регіон']!='Україна']

lines = alt.Chart(__data).mark_line().encode(
    x = alt.X('max(value):Q', title=''),
    x2 = alt.X2('min(value):Q', title=''),
    y = alt.Y('variable:N', 
     sort=alt.Sort(field='value', op='max', order='descending')
            )
    )

points = alt.Chart(__data).mark_circle(size = 80).encode(
    x = alt.X('value:Q'),
    y = alt.Y('variable:N',
     sort=alt.Sort(field='value', op='max', order='descending')
     ),
    color  = alt.Color('рік:N')
)
__data_total = df_tall[df_tall['Регіон']=='Україна']

lines_total = alt.Chart(__data_total).mark_line().encode(
    y = alt.Y('max(value):Q', title=''),
    y2 = alt.Y2('min(value):Q', title=''),
    x = alt.X('variable:N', 
     sort=alt.Sort(field='value', op='max', order='descending')
            )
    )

points_total = alt.Chart(__data_total).mark_circle(size = 80).encode(
    y = alt.Y('value:Q'),
    x = alt.X('variable:N',
     sort=alt.Sort(field='value', op='max', order='descending')
     ),
    color  = alt.Color('рік:N')
)
alt.layer(lines, points).properties(width=700/5, height=100).facet(
                facet=alt.Facet('Регіон:N'), columns=5
          )|alt.layer(lines_total, points_total).properties(width=200, height=500)

alt.HConcatChart(...)